In [ ]:
# Storing chat history in a database instead of sending the entire conversation to the API 
# is much more cost-effective. However, if a database is not an option, we can use this approach 
# to remove older messages and keep only the most recent ones to reduce token usage.

In [1]:
%pip install tiktoken openai tenacity --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import getpass
from openai import OpenAI
import tiktoken
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

secret_key = getpass.getpass('Please enter your openai key:')

# Set up your OpenAI client
client = OpenAI(api_key=secret_key)

In [9]:
# Exponentially increasing wait time between retries.  
# The first retry waits at least 1 second, then the wait time doubles (e.g., 1s → 2s → 4s → 8s → 16s).  
# The maximum wait time is capped at 60 seconds.  
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def chatgpt_completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

In [10]:
# Converts user input or AI response into a structured chat history format.
# If `user_prompt` is provided, it returns a dictionary representing a user message.
#  Otherwise, it extracts and returns the AI's response from the API response object.
def parse_for_chat_history(response: client = False, user_prompt=None):
    if user_prompt:
        return {'role': 'user', 'content': user_prompt}
    return {'role': 'assistant', 'content': response.choices[0].message.content}

In [11]:
  
# Calculates the total number of tokens in a list of messages for OpenAI GPT models.

 #   - Uses `tiktoken` to encode messages.
 #   - Applies model-specific tokenization rules.
 #   - Handles unknown models by falling back to known versions.
 #   - Raises an error if the model is not supported.
def num_tokens_from_messages(messages, model="gpt-4o-mini"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0301",
        "gpt-3.5-turbo",
        "gpt-4o-mini",
        "gpt-4o-mini-2024-07-18",
        "gpt-4o"
    }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = (
            4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        )
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print(
            "Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613."
        )
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301")
    elif "gpt-4" in model:
        print(
            "Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613."
        )
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    
    # Token counting
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message # Base tokens per message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value)) # Encode and count tokens
            if key == "name":
                num_tokens += tokens_per_name  # Adjust if "name" exist
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [12]:
messages = [
  {"role": "system", "content": "You are an expert assistant skilled at simplifying complex technical concepts."},
  {"role": "system", "name": "example_user", "content": "We need to optimize our server infrastructure for scalability and performance."},
  {"role": "system", "name": "example_assistant", "content": "We should focus on improving server resources to handle more traffic and ensure faster load times."},
  {"role": "system", "name": "example_user", "content": "We should also explore cloud solutions to better manage fluctuating demand."},
  {"role": "system", "name": "example_assistant", "content": "Using cloud services will help us scale dynamically based on user demand."},
  {"role": "user", "content": "Let's start by migrating the most critical services to the cloud and monitor the performance."},
]

model = "gpt-4o-mini"

print(f"{num_tokens_from_messages(messages, model)} prompt tokens counted.")
# Should show ~127 total_tokens

124 prompt tokens counted.


In [13]:
article_headings = [
    "I. Introduction A. Definition of Programming Languages B. Overview of Their Evolution C. Importance of Understanding Their Development",
    "II. Historical Background A. The Early Days of Computing B. The First Programming Languages C. The Rise of High-Level Languages",
    "III. Key Milestones in Programming A. The Introduction of Object-Oriented Programming B. The Shift to Functional Programming C. The Impact of Scripting Languages",
    "IV. The Role of Programming Paradigms A. Imperative vs. Declarative Programming B. The Evolution of Object-Oriented Programming C. The Growing Popularity of Functional Programming",
    "V. The Influence of Programming Languages on Software Development A. The Role of C and C++ in Systems Programming B. The Impact of Java and C# on Enterprise Applications C. The Rise of Python in AI and Data Science",
    "VI. Open Source and Community-Driven Development A. The Growth of Open Source Languages B. The Role of Developer Communities C. The Influence of GitHub and Open Collaboration",
    "VII. The Modern Programming Landscape A. The Popularity of JavaScript and Web Development B. The Expansion of Mobile Development with Swift and Kotlin C. The Role of Rust in Systems Programming",
    "VIII. The Impact of AI and Automation on Programming A. The Rise of AI-Assisted Code Generation B. The Influence of Machine Learning on Software Development C. The Future of Low-Code and No-Code Solutions",
    "IX. The Role of Cloud Computing and DevOps A. The Shift to Cloud-Native Development B. The Importance of DevOps and CI/CD Pipelines C. The Future of Serverless Computing",
    "X. Security and Programming Languages A. The Importance of Secure Coding Practices B. The Role of Memory-Safe Languages C. The Growing Focus on Cybersecurity in Development",
    "XI. Challenges in Modern Software Development A. The Complexity of Large-Scale Systems B. The Difficulty of Maintaining Legacy Code C. The Need for Better Developer Tools",
    "XII. The Future of Programming Languages A. The Impact of Quantum Computing B. The Role of Domain-Specific Languages C. The Evolution of Human-Readable Code",
    "XIII. Comparing Popular Programming Languages A. Python vs. Java for General-Purpose Development B. JavaScript vs. TypeScript for Web Applications C. Rust vs. C++ for Systems Programming",
    "XIV. The Economic and Social Impact of Programming A. The Demand for Skilled Developers B. The Role of Coding in Job Automation C. The Importance of Digital Literacy",
    "XV. Ethical Considerations in Software Development A. The Role of AI Ethics in Programming B. The Responsibility of Developers in Data Privacy C. The Impact of Open Source on Ethical Software Development",
    "XVI. The Influence of Hardware on Programming A. The Role of GPUs in Modern Computing B. The Shift to ARM Architecture C. The Impact of Edge Computing",
    "XVII. Conclusion A. Recap of the Evolution of Programming Languages B. The Importance of Continuous Learning C. Final Thoughts on the Future of Software Development",
    "XVIII. References A. List of Sources B. Additional Reading C. Further Research",
    "XIX. Glossary A. Key Terms B. Definitions",
    "XX. Appendix A. Timeline of Programming Language Evolution B. Code Examples from Different Paradigms C. Statistical Data on Programming Language Popularity",
]


In [14]:
system_prompt = "You are a knowledgeable assistant for a technology news website. You are writing a series of articles about the evolution of programming languages. You have been given a list of headings for each article. You need to write a short paragraph for each heading. You can use the headings as a starting point for your writing.\n\n"
# Add a section to list all subheadings to provide context.
system_prompt += "All of the subheadings:\n"

# Iterate through all the article headings and append them to the system message
for heading in article_headings:
    system_prompt += f"{heading}\n"

# Create the chat history object: 
chat_history =  []

# Add the system message as the first entry in the chat history.
# This ensures that the AI always has the necessary context when generating responses.
chat_history.append({'role': 'system', 'content': system_prompt})

In [15]:
# Ensures that if the token count exceeds the limit, the last message will be removed.  
# This helps manage the token count as the chat history expands.  
MAX_TOKEN_SIZE = 2048

# Iterate through all article headings and generate a response for each
for heading in article_headings:
    
    # Add on a user prompt to the chat history object:
    chat_history.append(
        {"role": "user", "content": f"Write a short paragraph about {heading}"}
    )

    # Request a response from ChatGPT using the current chat history
    response = chatgpt_completion_with_backoff(
        model="gpt-4o-mini", messages=chat_history
    )

    # Append ChatGPT's response to the chat history
    chat_history.append(parse_for_chat_history(response))

    # Whilst the Chat history object is more than 2048 tokens, remove the oldest non-system message:
    while num_tokens_from_messages(chat_history, model='gpt-4o-mini') > 2048:
        # Find the first non-system message in the chat history
        non_system_msg_index = next(
            (i for i, msg in enumerate(chat_history) if msg["role"] != "system"), None
        )

        # If there is a non-system message, remove it:
        if non_system_msg_index is not None:
            chat_history.pop(non_system_msg_index)
        
        print("Removed a message to reduce token count!")

    # Print the response:
    print(response)

ChatCompletion(id='chatcmpl-B6gJvMsg9FMMphJjPfBD3gNf2caHA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I. Introduction\n\nA. Definition of Programming Languages  \nProgramming languages are formal systems of communication designed for expressing algorithms and controlling the behavior of machines, particularly computers. They enable developers to write instructions that a computer can execute, serving as the foundation for software development and technological innovation. \n\nB. Overview of Their Evolution  \nThe evolution of programming languages began with simple assembly languages and has progressed through several major phases, including the introduction of high-level languages, the rise of object-oriented programming, and the advent of powerful scripting languages. Each phase has brought about new paradigms, enabling developers to write more efficient, readable, and accessible code, which in turn has transformed computing 